In [1]:
import pandas as pd
import numpy as np
from constants import *

import requests
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random

from helpers.scraper_helpers import parse_def, parse_gca, parse_gk, parse_misc, parse_pass, parse_passTypes, parse_poss, parse_shooting

In [21]:

teams = {}
years = list(range(2023, 2020, - 1))
standings_url = "https://fbref.com/en/comps/Big5/Big-5-European-Leagues-Stats"
f = open("data/temp.txt", "w")

for year in years:
    data = requests.get(standings_url)
    time.sleep(random.randint(1, 3))
    print(data.status_code)
    soup = BeautifulSoup(data.text, features="html.parser")
    standings_table = soup.select('table.stats_table')[0]
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"

    links = [l.get("href") for l in standings_table.find_all('a') if '/squads/' in l.get("href")]
    # links = [l for l in links if '/squads/' in l]

    if year == years[0]:
        teams = {x.split("/")[-1] for x in links}
    links = [x for x in links if x.split("/")[-1] in teams]

    f.write("-----" + str(year) + "-----\n")
    for l in links:
        name = l.split("/")[-1]
        name = name.split("-")[:-1]
        name = " ".join(name)
        f.write(name + "\n")
    

f.close()
    

200
200
200


In [5]:
HEADERS = [
        {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36"},
        {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:93.0) Gecko/20100101 Firefox/93.0"},
        {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36"},
        {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Safari/605.1.15"},
        {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"},
        {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:92.0) Gecko/20100101 Firefox/92.0"},
        {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36"},
        {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"},
        {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:93.0) Gecko/20100101 Firefox/93.0"},
        {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:92.0) Gecko/20100101 Firefox/92.0"},
        {"User-Agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36"},
        {"User-Agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"},
        {"User-Agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:93.0) Gecko/20100101 Firefox/93.0"},
        {"User-Agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:92.0) Gecko/20100101 Firefox/92.0"},
        {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_5_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36"},
        {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_5_2) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.2 Safari/605.1.15"},
        {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_5_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"},
        {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_5_2) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Safari/605.1.15"},
        {"User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36"},
        {"User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"},
        {"User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:93.0) Gecko/20100101 Firefox/93.0"},
        {"User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:92.0) Gecko/20100101 Firefox/92.0"},
        {"User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 15_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.0 Mobile/15E148 Safari/604.1"},
        {"User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 15_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.0 Mobile/15E148 Safari/604.1"},
        {"User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 15_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.0 Mobile/15E148 Safari/604.1"},
        {"User-Agent": "Mozilla/5.0 (Linux; Android 12; Pixel 6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Mobile Safari/537.36"},
        {"User-Agent": "Mozilla/5.0 (Linux; Android 12; Pixel 6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Mobile Safari/537.36"},
        {"User-Agent": "Mozilla/5.0 (Linux; Android 12; Pixel 6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Mobile Safari/537.36"},
        {"User-Agent": "Mozilla/5.0 (Linux; Android 12; Pixel 6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Mobile Safari/537.36"},
        {"User-Agent": "Mozilla/5.0 (Linux; Android 12; Pixel 6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Mobile Safari/537.36"},
        {"User-Agent": "Mozilla/5.0 (Linux; Android 12; Pixel 6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Mobile Safari/537.36"}
    ]

In [2]:
def parse_url(url, parse, text):
    while True:
        data = requests.get(url, headers=random.choice(HEADERS))
        if (data.status_code == 200):
            df = pd.read_html(data.text, match=text)[0]
            df = parse(df)
            return df
        else:
            print(f"Error: {data.status_code}")
            time.sleep(10)

In [12]:
# parse_fcns = [parse_def, parse_gca, parse_gk, parse_misc, parse_pass, parse_passTypes, parse_poss, parse_shooting]
# table_names = ["Defensive Actions", "Goal and Shot Creation", "Goalkeeping", "Miscellaneous Stats", "Passing", "Pass Types", "Possession", "Shooting"]
parse_fcns = [parse_def, parse_gca, parse_misc, parse_pass, parse_poss, parse_shooting]
table_names = ["Defensive Actions", "Goal and Shot Creation", "Miscellaneous Stats", "Passing", "Possession", "Shooting"]
years = list(range(2023, 2020, - 1))
all_matches = []
standings_url = "https://fbref.com/en/comps/Big5/Big-5-European-Leagues-Stats"
teams = {}
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text, features="html.parser")
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a') if '/squads/' in l.get("href")]

    if year == years[0]:
        teams = {x.split("/")[-1] for x in links}
        print(teams)

    team_urls = [f"https://fbref.com{l}" for l in links if l.split("/")[-1] in teams]

    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
        
    # Get team data
    # team_urls = [x for x in team_urls if 'Tottenham' in x]

    for team_url in team_urls:
        # Get match data
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        time.sleep(3)

        print(team_name + " " + str(year))
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        matches = matches.drop(columns=["Poss", "xG", "Attendance", "Captain", "Match Report", "Referee", "Notes"])
                
        # Get stats links
        soup = BeautifulSoup(data.text, features="html.parser")
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and ("all_comps/shooting/" in l in l or "all_comps/passing/" in l
                                         in l or "all_comps/gca/" in l or "all_comps/defense/" in l
                                        or "all_comps/possession/" in l or "all_comps/misc/" in l)]
        links = list(set(links))
        links = [f"https://fbref.com{l}" for l in links]
        links.sort()
                
        dfs = []
        print(links)
        break
        for url, category, parser in zip(links, table_names, parse_fcns):
            dfs.append(parse_url(url, parser, category))
            time.sleep(random.randint(1, 3))
            
        team_data = matches
        for df in dfs:
            try:
                team_data = team_data.merge(df, how="left", on="Date")
            except ValueError:
                continue
        
        team_data = team_data[(team_data["Comp"] == "Premier League")
                              | (team_data["Comp"] == "Bundesliga")
                              | (team_data["Comp"] == "La Liga")
                              | (team_data["Comp"] == "Serie A")
                              | (team_data["Comp"] == "Ligue 1")
                              ]
            
        # Add year and team name
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        break
    break
    
# matches_df = pd.concat(all_matches)
# matches_df.columns = [c.lower() for c in matches_df.columns]

# matches_df["date"] = pd.to_datetime(matches_df["date"])

{'Brest-Stats', 'Lens-Stats', 'Roma-Stats', 'Wolverhampton-Wanderers-Stats', 'Paris-Saint-Germain-Stats', 'West-Ham-United-Stats', 'Montpellier-Stats', 'Torino-Stats', 'Newcastle-United-Stats', 'Augsburg-Stats', 'Strasbourg-Stats', 'Bochum-Stats', 'Chelsea-Stats', 'Bologna-Stats', 'Real-Madrid-Stats', 'Monza-Stats', 'Clermont-Foot-Stats', 'Koln-Stats', 'Frosinone-Stats', 'Valencia-Stats', 'Bayern-Munich-Stats', 'Lecce-Stats', 'Lorient-Stats', 'Rennes-Stats', 'Sassuolo-Stats', 'Le-Havre-Stats', 'Monchengladbach-Stats', 'Burnley-Stats', 'Heidenheim-Stats', 'Aston-Villa-Stats', 'Manchester-United-Stats', 'Celta-Vigo-Stats', 'Hellas-Verona-Stats', 'Lyon-Stats', 'Alaves-Stats', 'Cadiz-Stats', 'Real-Sociedad-Stats', 'Atalanta-Stats', 'Liverpool-Stats', 'Dortmund-Stats', 'Empoli-Stats', 'Luton-Town-Stats', 'Getafe-Stats', 'Arsenal-Stats', 'Crystal-Palace-Stats', 'Brentford-Stats', 'Milan-Stats', 'Granada-Stats', 'Rayo-Vallecano-Stats', 'Sheffield-United-Stats', 'Brighton-and-Hove-Albion-Stats

In [7]:
matches_df.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,fk,pk,pkattfor,xg,npxg,npxg/sh,g-xg,np:g-xg,season,team
0,2023-08-13,14:00,Premier League,Matchweek 1,Sun,Away,D,2,2,Brentford,...,0.0,0.0,0.0,1.3,1.3,0.07,0.7,0.7,2023,Tottenham Hotspur
1,2023-08-19,17:30,Premier League,Matchweek 2,Sat,Home,W,2,0,Manchester Utd,...,0.0,0.0,0.0,1.7,1.7,0.10,-0.7,-0.7,2023,Tottenham Hotspur
2,2023-08-26,12:30,Premier League,Matchweek 3,Sat,Away,W,2,0,Bournemouth,...,1.0,0.0,0.0,2.1,2.1,0.12,-0.1,-0.1,2023,Tottenham Hotspur
4,2023-09-02,15:00,Premier League,Matchweek 4,Sat,Away,W,5,2,Burnley,...,0.0,0.0,0.0,2.2,2.2,0.11,2.8,2.8,2023,Tottenham Hotspur
5,2023-09-16,15:00,Premier League,Matchweek 5,Sat,Home,W,2,1,Sheffield Utd,...,0.0,0.0,0.0,2.1,2.1,0.08,-0.1,-0.1,2023,Tottenham Hotspur
